# Solar Power Prediction - Neural Network

This notebook implements a neural network model for predicting solar power output using PyTorch.

**Model:**
- Feedforward Neural Network with GPU acceleration


In [ ]:

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

DATASET_PATH = Path('../../data/data-3/timeseries/61272.csv')

data = pd.read_csv(DATASET_PATH)

X, y = train_test_split(data, test_size=0.2, random_state=42)
X.sort_index(inplace=True)

X.drop(['time', 'direct_normal_irradiance', 'diffuse_radiation', 'wind_speed_80m', 'wind_speed_180m', 'weather_code'], axis=1, inplace=True)
X.head()

# corr_mat = X.corr()
# print(corr_mat)


In [ ]:
# Separate features and target variable
X_features = X.drop('power', axis=1)
y_target = X['power']

print("Features used for regression:")
print(list(X_features.columns))
print(f"\nNumber of samples: {len(X_features)}")
print(f"Number of features: {X_features.shape[1]}")


In [ ]:
# Apply feature scaling using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

# Convert back to DataFrame for easier interpretation
X_scaled_df = pd.DataFrame(X_scaled, columns=X_features.columns, index=X_features.index)

print("="*70)
print("FEATURE SCALING APPLIED")
print("="*70)
print("\nOriginal feature statistics:")
print(X_features.describe())
print("\n" + "="*70)
print("Scaled feature statistics (mean≈0, std≈1):")
print(X_scaled_df.describe())


## Neural Network Regression

Use a feedforward neural network to capture complex non-linear relationships between features and power output.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if CUDA is available
assert torch.cuda.is_available()
device = torch.device('cuda')


In [ ]:
# Define a simple feedforward neural network
class SolarPowerNet(nn.Module):
    def __init__(self, input_size):
        super(SolarPowerNet, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(32, 16),
            nn.ReLU(),

            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.network(x)

# Initialize model
input_size = X_scaled.shape[1]
model = SolarPowerNet(input_size).to(device)

print("="*70)
print("NEURAL NETWORK ARCHITECTURE")
print("="*70)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Input features: {input_size}")
print(f"Output: 1 (power)")


In [ ]:
# Training setup
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X_scaled)  # Keep on CPU for DataLoader
y_tensor = torch.FloatTensor(y_target.values).reshape(-1, 1)  # Keep on CPU for DataLoader

# Create DataLoader
dataset = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Dataset size: {len(dataset)}")
print(f"Batch size: 32")
print(f"Number of batches: {len(train_loader)}")


In [ ]:
# Training loop
num_epochs = 100
train_losses = []

# Clear CUDA cache to avoid memory issues
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU Memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"GPU Memory reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

print("="*70)
print("TRAINING NEURAL NETWORK")
print("="*70)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_X, batch_y in train_loader:
        # Move batch to device
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("\nTraining complete!")


In [ ]:
# Plot training loss
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(train_losses, linewidth=2, color='blue')
ax.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax.set_ylabel('Loss (MSE)', fontsize=12, fontweight='bold')
ax.set_title('Neural Network Training Loss', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Evaluate model
model.eval()
with torch.no_grad():
    # Move input tensor to the same device as the model (CUDA)
    y_pred_nn_tensor = model(X_tensor.to(device))
    # Move to CPU first, then detach from computation graph, then convert to numpy
    # Using tolist() to avoid numpy compatibility issues with older PyTorch versions
    y_pred_nn = np.array(y_pred_nn_tensor.detach().cpu().tolist()).flatten()

# Convert y_target to numpy for metrics calculation
y_target_np = y_target.values

# Calculate performance metrics
mse_nn = mean_squared_error(y_target_np, y_pred_nn)
rmse_nn = np.sqrt(mse_nn)
mae_nn = mean_absolute_error(y_target_np, y_pred_nn)
r2_nn = r2_score(y_target_np, y_pred_nn)

print("="*70)
print("NEURAL NETWORK RESULTS")
print("="*70)
print(f"\nModel Performance Metrics:")
print(f"  R² Score:                 {r2_nn:.6f}")
print(f"  Mean Squared Error (MSE): {mse_nn:.2f}")
print(f"  Root Mean Squared Error:  {rmse_nn:.2f}")
print(f"  Mean Absolute Error:      {mae_nn:.2f}")

print("\n" + "="*70)
print(f"{'Model':<20} {'R²':<12} {'RMSE':<12} {'MAE':<12}")
print("-"*70)
print(f"{'Neural Network':<20} {r2_nn:<12.6f} {rmse_nn:<12.2f} {mae_nn:<12.2f}")
print("="*70)

# Calculate improvements

In [ ]:
# Visualize neural network performance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot: Actual vs Predicted (Neural Network)
axes[0].scatter(y_target_np, y_pred_nn, alpha=0.5, s=10, label='Neural Network', c='purple')
axes[0].plot([y_target_np.min(), y_target_np.max()],
             [y_target_np.min(), y_target_np.max()],
             'r--', lw=2, label='Perfect prediction')
axes[0].set_xlabel('Actual Power', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Power', fontsize=12, fontweight='bold')
axes[0].set_title(f'Neural Network: Actual vs Predicted Power\n(R² = {r2_nn:.4f})',
                  fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot (Neural Network)
residuals_nn = y_target_np - y_pred_nn
axes[1].scatter(y_pred_nn, residuals_nn, alpha=0.5, s=10, c='purple')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Power', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[1].set_title(f'Neural Network: Residual Plot\n(RMSE = {rmse_nn:.2f})',
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
